# Lecture 8: Making Decisions - Markov Decision Processes (MDPs)

## Overview
บทบรรยายนี้แสดงการประยุกต์ใช้ทฤษฎี **Markov Decision Process (MDP)** ในการแก้ปัญหาการตัดสินใจภายใต้ความไม่แน่นอน

### หัวข้อหลัก:
1. **Grid World Environment** - สภาพแวดล้อมจำลองที่มีความไม่แน่นอน
2. **Value Iteration** - อัลกอริทึมสำหรับหาค่าที่เหมาะสมของสถานะ
3. **Policy Iteration** - อัลกอริทึมสำหรับหา policy ที่เหมาะสมโดยตรง
4. **Bellman Equation** - สมการพื้นฐานสำหรับคำนวณค่าของสถานะ

In [1]:
import numpy as np

In [3]:
class GridWorld:
    def __init__(self, height=3, width=4):
        self.height = height
        self.width = width
        self.grid = np.zeros((height, width))
        self.terminal_states = {(0, 3): 1, (1, 3): -1}  # (state): reward
        self.living_reward = -0.04
        self.gamma = 1.0
        self.p_intended = 0.8
        self.p_perpendicular = 0.1  # For each perpendicular direction
        self.actions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        
    def is_valid_state(self, state):
        row, col = state
        if row < 0 or row >= self.height or col < 0 or col >= self.width:
            return False
        if (row, col) == (1, 1):  # Wall
            return False
        return True
    
    def get_transition_probs(self, state, action):
        if state in self.terminal_states:
            return [(state, 1.0)]
        
        transitions = []

        perp1 = (action[1], action[0])    # Rotate 90° clockwise
        perp2 = (-action[1], -action[0])  # Rotate 90° counterclockwise
        
        for next_action, prob in [(action, self.p_intended), 
                                  (perp1, self.p_perpendicular),
                                  (perp2, self.p_perpendicular)]:
            next_state = (state[0] + next_action[0], state[1] + next_action[1])
            if self.is_valid_state(next_state):
                transitions.append((next_state, prob))
            else:
                transitions.append((state, prob))  # Stay in current state
                
        return transitions
    
    def get_reward(self, state):
        if state in self.terminal_states:
            return self.terminal_states[state]
        return self.living_reward

## 1. GridWorld Class - การจำลอง Markov Decision Process (MDP)

คลาส `GridWorld` นี้จำลองสภาพแวดล้อมตาม **Grid World** ที่อธิบายในบทบรรยาย โดยมีองค์ประกอบสำคัญของ MDP:

### องค์ประกอบของ MDP: $(\mathcal{S}, \mathcal{A}, P, R)$
- **$\mathcal{S}$ (States)**: ตำแหน่งต่างๆ ในตาราง 3×4
- **$\mathcal{A}$ (Actions)**: การเคลื่อนที่ 4 ทิศทาง [Right, Down, Left, Up]
- **$P$ (Transition Model)**: ความน่าจะเป็นการเปลี่ยนสถานะ (มีความไม่แน่นอน)
- **$R$ (Reward Function)**: รางวัลที่ได้รับในแต่ละสถานะ

### ความไม่แน่นอนในการเคลื่อนที่ (Stochastic Actions):
- **80%** ไปในทิศทางที่ตั้งใจ (`p_intended = 0.8`)
- **10%** ไปในทิศทางตั้งฉากด้านหนึ่ง (`p_perpendicular = 0.1`)
- **10%** ไปในทิศทางตั้งฉากอีกด้านหนึ่ง

### ค่ารางวัล:
- **Living reward**: -0.04 (รางวัลเล็กน้อยในแต่ละขั้นตอน)
- **Terminal states**: +1 ที่ตำแหน่ง (0,3), -1 ที่ตำแหน่ง (1,3)

## 2. Value Iteration Algorithm

**Value Iteration** เป็นอัลกอริทึมสำหรับหาค่าที่เหมาะสมของแต่ละสถานะ $V(s)$ โดยใช้ **Bellman Update**:

$$V_{i+1}(s) := R(s) + \gamma \max_a \sum_{s'} P(s'|s,a) V_i(s')$$

### วิธีการทำงาน:
1. **เริ่มต้น**: กำหนดค่า $V_0(s) = 0$ สำหรับทุกสถานะ
2. **อัพเดท**: คำนวณค่า $V$ ใหม่สำหรับทุกสถานะพร้อมกัน
3. **ทำซ้ำ**: จนกว่าจะลู่เข้า (convergence)

### ขั้นตอนในโค้ด:
- คำนวณ **Q-value** สำหรับแต่ละ action: $Q(s,a) = \sum_{s'} P(s'|s,a) V(s')$
- หาค่าสูงสุด: $\max_a Q(s,a)$
- อัพเดทค่า: $V_{new}(s) = R(s) + \gamma \max_a Q(s,a)$

In [4]:
def value_iteration(grid, threshold=1e-3):
    # Initialize values
    V = {(i, j): 0 for i in range(grid.height) for j in range(grid.width) 
            if grid.is_valid_state((i, j))}
    
    iteration = 0
    while True:
        biggest_change = 0
        V_new = V.copy()
        
        # Update each state
        for state in V:
            if state in grid.terminal_states:
                V_new[state] = grid.get_reward(state)
                
            else:
                # Calculate max_a \sum_{s'} P(s'|s,a) V(s')
                max_q = float('-inf')

                for action in grid.actions:
                    q = 0
                    for next_state, prob in grid.get_transition_probs(state, action):
                        q += prob * V[next_state]
                    max_q = max(max_q, q)
                
                V_new[state] = grid.get_reward(state) + grid.gamma * max_q
            biggest_change = max(biggest_change, abs(V_new[state] - V[state]))
        
        V = V_new
        iteration += 1
        
        # Check convergence
        if biggest_change < threshold:
            break
            
    return V, iteration

In [5]:

grid = GridWorld()
V, iterations = value_iteration(grid)

print(f"\nConverged after {iterations} iterations")
print("\nFinal values:")
for i in range(grid.height):
    for j in range(grid.width):
        if not grid.is_valid_state((i, j)):
            print("   XXXXX ", end="")
        else:
            print(f" {V[(i, j)]:7.3f} ", end="")
    print()


Converged after 20 iterations

Final values:
   0.812    0.868    0.918    1.000 
   0.762    XXXXX    0.660   -1.000 
   0.705    0.655    0.611    0.387 


In [6]:
def policy_extraction(grid, V):
    policy = {state: None for state in V}
    
    for state in V:
        if state in grid.terminal_states:
            policy[state] = None
        
        else:
            max_q = float('-inf')
            best_action = None
            for action in grid.actions:
                q = 0
                for next_state, prob in grid.get_transition_probs(state, action):
                    q += prob * V[next_state]
                if q > max_q:
                    max_q = q
                    best_action = action
            policy[state] = best_action
        
    return policy

policy = policy_extraction(grid, V)
policy


{(0, 0): (0, 1),
 (0, 1): (0, 1),
 (0, 2): (0, 1),
 (0, 3): None,
 (1, 0): (-1, 0),
 (1, 2): (-1, 0),
 (1, 3): None,
 (2, 0): (-1, 0),
 (2, 1): (0, -1),
 (2, 2): (0, -1),
 (2, 3): (0, -1)}

## 3. Policy Extraction - การสกัด Optimal Policy

หลังจากได้ค่า $V(s)$ ที่เหมาะสมแล้ว เราสามารถสกัด **Optimal Policy** ได้โดยใช้หลักการ **Maximum Expected Utility**:

$$\pi^*(s) = \arg \max_a \sum_{s'} P(s'|s,a) V(s')$$

### ความหมาย:
- สำหรับแต่ละสถานะ $s$ ให้เลือก action $a$ ที่ทำให้ได้ค่า expected utility สูงสุด
- นี่คือการใช้ **one-step lookahead** เพื่อเลือก action ที่ดีที่สุด

### ในโค้ด:
- คำนวณ Q-value สำหรับทุก action
- เลือก action ที่ให้ค่า Q สูงสุด
- นั่นคือ optimal action สำหรับสถานะนั้น

In [7]:
def print_policy(grid, policy):
    for i in range(grid.height):
        for j in range(grid.width):
            if not grid.is_valid_state((i, j)):
                print("\tX", end="")
            else:
                if policy[(i, j)] == (0, 1):
                    print("\t\u2192", end="")
                elif policy[(i, j)] == (1, 0):
                    print("\t\u2193", end="")
                elif policy[(i, j)] == (0, -1):
                    print("\t\u2190", end="")
                elif policy[(i, j)] == (-1, 0):
                    print("\t\u2191", end="")
                else:
                    print("\t*", end="")
        print()

print_policy(grid, policy)

	→	→	→	*
	↑	X	↑	*
	↑	←	←	←


## 4. Policy Visualization

ฟังก์ชัน `print_policy` แสดง optimal policy ในรูปแบบที่เข้าใจง่าย:
- **→** (Right): เคลื่อนที่ไปทางขวา
- **↓** (Down): เคลื่อนที่ไปทางล่าง  
- **←** (Left): เคลื่อนที่ไปทางซ้าย
- **↑** (Up): เคลื่อนที่ไปทางบน
- **X**: กำแพง/สิ่งกีดขวาง
- **\***: สถานะสิ้นสุด (terminal states)

### ความสำคัญของ Living Reward:
ค่า living reward (-0.04) มีผลต่อ optimal policy:
- **ค่าลบ**: agent จะพยายามไปถึงเป้าหมายเร็วที่สุด
- **ค่าบวก**: agent อาจจะเดินอ้อมเพื่อได้รางวัลมากขึ้น

## 5. Policy Iteration Algorithm

**Policy Iteration** เป็นอีกวิธีหนึ่งในการหา optimal policy โดยตรง แทนที่จะหาค่า V ก่อน

### ข้อดีของ Policy Iteration:
- มักจะ **converge เร็วกว่า** Value Iteration
- หา policy โดยตรงแทนที่จะต้องสกัดจาก V
- เหมาะสำหรับปัญหาที่มี action space ใหญ่

### อัลกอริทึมทำงาน 2 ขั้นตอนสลับกัน:

#### 1. Policy Evaluation
คำนวณ $V^{\pi}(s)$ สำหรับ policy ปัจจุบัน โดยแก้สมการ Bellman แบบเชิงเส้น:
$$V^{\pi}(s) = R(s) + \gamma \sum_{s'} P(s'|s,\pi(s)) V^{\pi}(s')$$

#### 2. Policy Improvement  
ปรับปรุง policy ให้ดีขึ้นโดยใช้ one-step lookahead:
$$\pi_{new}(s) = \arg \max_a \sum_{s'} P(s'|s,a) V^{\pi}(s')$$

### หยุดทำงานเมื่อ:
Policy ไม่เปลี่ยนแปลงอีก (policy stable)

In [7]:
def policy_evaluation(grid, policy, updates=20):
    V = {state: 0 for state in policy}

    for _ in range(updates):
        V_new = V.copy()
        for state in V:
            if state in grid.terminal_states:
                V_new[state] = grid.get_reward(state)

            else:
                action = policy[state]
                q = 0
                for next_state, prob in grid.get_transition_probs(state, action):
                    q += prob * V[next_state]
                V_new[state] = grid.get_reward(state) + grid.gamma * q

        V = V_new

    return V


def policy_iteration(grid, threshold=1e-3):
    # Initialize policy
    policy = {(i, j): grid.actions[0] for i in range(grid.height) for j in range(grid.width) 
            if grid.is_valid_state((i, j))}
    
    iteration = 0
    while True:
        # Policy evaluation
        V = policy_evaluation(grid, policy)
        
        # Policy improvement
        policy_stable = True
        for state in V:
            old_action = policy[state]
            max_q = float('-inf')
            best_action = None

            for action in grid.actions:
                q = 0
                for next_state, prob in grid.get_transition_probs(state, action):
                    q += prob * V[next_state]
                if q > max_q:
                    max_q = q
                    best_action = action
                    
            policy[state] = best_action
            if best_action != old_action:
                policy_stable = False
        
        iteration += 1
        if policy_stable:
            break
            
    return policy, V, iteration

### Policy Evaluation Function

ฟังก์ชัน `policy_evaluation` คำนวณค่า $V^{\pi}(s)$ สำหรับ policy ที่กำหนด:

**ความแตกต่างจาก Value Iteration:**
- **ไม่มี max operator** เพราะ action ถูกกำหนดโดย policy แล้ว
- กลายเป็น **ระบบสมการเชิงเส้น** ที่แก้ได้ง่าย
- ใช้ **simplified Bellman update**:

$$V_{i+1}(s) = R(s) + \gamma \sum_{s'} P(s'|s,\pi(s))V_i(s')$$

**ในโค้ด:**
- `action = policy[state]` - ใช้ action ที่ policy กำหนด
- ไม่มีการหา max ของ Q-values
- ทำการ update แบบ iterative จนกว่าจะ converge

In [8]:
policy, V, iterations = policy_iteration(grid)

print(f"\nConverged after {iterations} iterations")
print("\nFinal values:")
for i in range(grid.height):
    for j in range(grid.width):
        if not grid.is_valid_state((i, j)):
            print("   XXXXX ", end="")
        else:
            print(f" {V[(i, j)]:7.3f} ", end="")
    print()


Converged after 3 iterations

Final values:
   0.812    0.868    0.918    1.000 
   0.762    XXXXX    0.660   -1.000 
   0.705    0.655    0.611    0.387 


## 6. สรุปและเปรียบเทียบ Value Iteration vs Policy Iteration

### ผลลัพธ์:
ทั้งสองอัลกอริทึมจะให้ **optimal policy เดียวกัน** และ **ค่า V เดียวกัน**

### ข้อแตกต่าง:

| แง่มุม | Value Iteration | Policy Iteration |
|--------|----------------|-----------------|
| **วิธีการ** | หาค่า V ก่อน แล้วสกัด policy | หา policy โดยตรง |
| **ความเร็ว** | อาจช้ากว่า | มักเร็วกว่า |
| **ความซับซ้อน** | ง่ายกว่า | ซับซ้อนกว่า |
| **การ update** | ทุกสถานะพร้อมกัน | สลับระหว่าง evaluation และ improvement |

### การเลือกใช้:
- **Value Iteration**: เหมาะสำหรับปัญหาขนาดเล็ก หรือเมื่อต้องการ implementation ที่ง่าย
- **Policy Iteration**: เหมาะสำหรับปัญหาขนาดใหญ่ที่ต้องการความเร็ว

### Bellman Equation - หัวใจของทั้งสองอัลกอริทึม:
$$V(s) = R(s) + \gamma \max_a \sum_{s'} P(s'|s,a) V(s')$$

สมการนี้เชื่อมโยงค่าของสถานะปัจจุบันกับค่าของสถานะถัดไป ทำให้เราหา optimal policy ได้